In [85]:
import pandas as pd

In [86]:
games = pd.read_csv("data.csv", index_col=0)

In [87]:
games.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,15.0,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,12.6,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.6,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,23.0,8.0,16.6,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,15.3,1.0,0.0,0.0,2022,Arsenal


In [88]:
#Checking for any values missing from the data (All expected is present)
games["team"].value_counts()

Arsenal                     44
Newcastle United            44
Leicester City              44
West Ham United             44
Aston Villa                 44
Everton                     44
Crystal Palace              44
Wolverhampton Wanderers     44
Manchester City             44
Southampton                 44
Leeds United                44
Brentford                   44
Liverpool                   44
Chelsea                     44
Manchester United           44
Brighton and Hove Albion    44
Tottenham Hotspur           44
Burnley                     38
Watford                     38
Norwich City                38
Fulham                       6
Bournemouth                  6
Nottingham Forest            6
Name: team, dtype: int64

In [89]:
games = pd.read_csv("data.csv", index_col=0)
games.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [90]:
del games["comp"]
del games["notes"]

In [91]:
games["date"] = pd.to_datetime(games["date"])
games["target"] = (games["result"] == "W").astype("int")

In [92]:
games["venue_code"] = games["venue"].astype("category").cat.codes
games["opp_code"] = games["opponent"].astype("category").cat.codes
games["hour"] = games["time"].str.replace(":.+", "", regex=True).astype("int")
games["day_code"] = games["date"].dt.dayofweek


In [93]:
from sklearn.ensemble import RandomForestClassifier

In [94]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [95]:
train = games[games["date"] < '2022-01-01']
test = games[games["date"] > '2022-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [96]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [97]:
preds = rf.predict(test[predictors])

In [98]:
from sklearn.metrics import accuracy_score

In [99]:
acc = accuracy_score(test["target"], preds)

In [100]:
acc

0.5984251968503937

In [101]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [102]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,242,69
1,135,62


In [103]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.4732824427480916

In [104]:
grouped_games = games.groupby("team")

In [105]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [106]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [107]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
4,2021-09-11,15:00,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,...,15,5,3.333333,0.333333,19.666667,6.000000,16.166667,0.666667,0.000000,0.000000
6,2021-09-18,15:00,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,...,15,5,3.666667,0.000000,22.000000,7.333333,15.200000,0.333333,0.000000,0.000000
8,2021-09-25,12:30,Matchweek 6,Sat,Away,W,1.0,0.0,Chelsea,1.7,...,12,5,2.000000,0.000000,22.000000,6.333333,14.666667,0.333333,0.000000,0.000000
10,2021-10-03,16:30,Matchweek 7,Sun,Away,D,2.0,2.0,Liverpool,1.2,...,16,6,0.666667,0.000000,18.666667,4.000000,15.400000,0.333333,0.000000,0.000000
11,2021-10-16,15:00,Matchweek 8,Sat,Home,W,2.0,0.0,Burnley,1.9,...,15,5,1.000000,0.666667,14.333333,2.333333,16.333333,0.666667,0.000000,0.000000
13,2021-10-23,17:30,Matchweek 9,Sat,Away,W,4.0,1.0,Brighton,3.9,...,17,5,1.666667,0.666667,14.333333,4.000000,15.966667,0.333333,0.000000,0.000000
15,2021-10-30,15:00,Matchweek 10,Sat,Home,L,0.0,2.0,Crystal Palace,1.1,...,15,5,2.666667,1.000000,17.000000,7.333333,15.700000,0.666667,0.000000,0.000000
17,2021-11-06,12:30,Matchweek 11,Sat,Away,W,2.0,0.0,Manchester Utd,2.0,...,12,5,2.000000,1.000000,17.666667,7.333333,15.766667,0.333333,0.000000,0.000000
18,2021-11-21,14:00,Matchweek 12,Sun,Home,W,3.0,0.0,Everton,2.3,...,14,6,2.000000,1.000000,17.666667,7.000000,15.766667,0.333333,0.000000,0.000000
20,2021-11-28,14:00,Matchweek 13,Sun,Home,W,2.0,1.0,West Ham,2.1,...,14,6,1.666667,0.666667,15.666667,5.000000,15.666667,0.333333,0.000000,0.000000


In [108]:
games_rolling = games.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [109]:
games_rolling

date   time        round  day venue result  \
team                                                                         
Arsenal                 4 2021-09-11  15:00  Matchweek 4  Sat  Home      W   
                        5 2021-09-18  15:00  Matchweek 5  Sat  Away      W   
                        7 2021-09-26  16:30  Matchweek 6  Sun  Home      W   
                        8 2021-10-02  17:30  Matchweek 7  Sat  Away      D   
                        9 2021-10-18  20:00  Matchweek 8  Mon  Home      D   
...                              ...    ...          ...  ...   ...    ...   
Wolverhampton Wanderers 1 2022-08-13  15:00  Matchweek 2  Sat  Home      D   
                        2 2022-08-20  12:30  Matchweek 3  Sat  Away      L   
                        4 2022-08-28  14:00  Matchweek 4  Sun  Home      D   
                        5 2022-08-31  19:30  Matchweek 5  Wed  Away      D   
                        6 2022-09-03  15:00  Matchweek 6  Sat  Home      W   

                            gf   ga        opponent   xg  ...  hour  day_code  \
team                                                      ...                   
Arsenal                 4  1.0  0.0    Norwich City  2.8  ...    15         5   
                        5  1.0  0.0         Burnley  1.2  ...    15         5   
                        7  3.0  1.0       Tottenham  0.8  ...    16         6   
                        8  0.0  0.0        Brighton  0.5  ...    17         5   
                        9  2.0  2.0  Crystal Palace  1.4  ...    20         0   
...                        ...  ...             ...  ...  ...   ...       ...   
Wolverhampton Wanderers 1  0.0  0.0          Fulham  0.7  ...    15         5   
                        2  0.0  1.0       Tottenham  0.8  ...    12         5   
                        4  1.0  1.0   Newcastle Utd  0.8  ...    14         6   
                        5  0.0  0.0     Bournemouth  1.5  ...    19         2   
                        6  1.0  0.0     Southampton  0.7  ...    15         5   

                           gf_rolling ga_rolling sh_rolling sot_rolling  \
team                                                                      
Arsenal                 4    0.000000   3.000000   9.666667    2.333333   
                        5    0.333333   2.333333  12.333333    3.000000   
                        7    0.666667   1.666667  14.666667    3.000000   
                        8    1.666667   0.333333  18.333333    5.333333   
                        9    1.333333   0.333333  11.000000    4.000000   
...                               ...        ...        ...         ...   
Wolverhampton Wanderers 1    1.000000   2.000000  13.000000    5.000000   
                        2    0.666667   1.666667   9.666667    4.000000   
                        4    0.333333   1.000000  14.000000    3.333333   
                        5    0.333333   0.666667  12.333333    2.666667   
                        6    0.333333   0.666667  15.666667    3.333333   

                          dist_rolling  fk_rolling  pk_rolling  pkatt_rolling  
team                                                                           
Arsenal                 4    14.600000    0.333333         0.0            0.0  
                        5    13.866667    0.333333         0.0            0.0  
                        7    14.766667    0.666667         0.0            0.0  
                        8    18.000000    0.666667         0.0            0.0  
                        9    18.966667    0.666667         0.0            0.0  
...                                ...         ...         ...            ...  
Wolverhampton Wanderers 1    17.666667    0.666667         0.0            0.0  
                        2    17.333333    0.333333         0.0            0.0  
                        4    19.333333    0.666667         0.0            0.0  
                        5    19.600000    1.000000         0.0            0.0  
                        6  

In [110]:
games_rolling.index = range(games_rolling.shape[0])

In [111]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [112]:
combined, precision = make_predictions(games_rolling, predictors + new_cols)

In [121]:
precision

0.5081967213114754

In [114]:
combined = combined.merge(games_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [115]:
combined.head(10)

,actual,predicted,date,team,opponent,result
17,0,0,2022-01-23,Arsenal,Burnley,D
18,1,0,2022-02-10,Arsenal,Wolves,W
19,1,0,2022-02-19,Arsenal,Brentford,W
20,1,0,2022-02-24,Arsenal,Wolves,W
21,1,1,2022-03-06,Arsenal,Watford,W
22,1,1,2022-03-13,Arsenal,Leicester City,W
23,0,1,2022-03-16,Arsenal,Liverpool,L
24,1,0,2022-03-19,Arsenal,Aston Villa,W
25,0,0,2022-04-04,Arsenal,Crystal Palace,L
26,0,0,2022-04-09,Arsenal,Brighton,L


In [122]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [117]:
combined["new_team"] = combined["team"].map(mapping)

In [118]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [119]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,0,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,0,0,2022-08-06,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
483,0,1,2022-08-20,Wolverhampton Wanderers,Tottenham,L,Wolves,1,1,Tottenham Hotspur,Wolves,W,Tottenham
484,0,0,2022-08-28,Wolverhampton Wanderers,Newcastle Utd,D,Wolves,0,0,Newcastle United,Wolves,D,Newcastle Utd
485,0,0,2022-08-31,Wolverhampton Wanderers,Bournemouth,D,Wolves,0,0,Bournemouth,Wolves,D,Bournemouth


In [120]:
#57/94 = 61% accuracy
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

1    57
0    37
Name: actual_x, dtype: int64